## process LFP from DR recordings

In [2]:
#Import packages
import pandas as pd
import h5py
import numpy as np
import scipy as sp
import scipy.signal as sg
import scipy.stats as st
import xarray as xr
import os
import matplotlib.pyplot as plt
from matplotlib import patches
import ast
import glob
import re

import sync.sync as sync
import probeSync
import ecephys

# from DR_analysis_utils import Session, makePSTH, make_neuron_time_trials_tensor 
# from DR_analysis_utils import compute_smoothed_response_rate, compute_block_modulation

%matplotlib notebook

In [3]:
mainpath=r"Y:\DRpilot_649943_20230215"
lfp_path=r"Y:\DRpilot_649943_20230215\DRpilot_649943_20230215_probeABCDEF\Record Node 103\experiment1\recording1\continuous\Neuropix-PXI-100.ProbeC-LFP\continuous.dat"
lfp_samples_path=r"Y:\DRpilot_649943_20230215\DRpilot_649943_20230215_probeABCDEF\Record Node 103\experiment1\recording1\continuous\Neuropix-PXI-100.ProbeC-LFP\sample_numbers.npy"


In [4]:
lfp_samples=np.load(lfp_samples_path)
lfp_samples

array([20539277, 20539278, 20539279, ..., 32567008, 32567009, 32567010],
      dtype=int64)

In [18]:
lfp_samples=lfp_samples-np.min(lfp_samples)
lfp_samples=lfp_samples*12

# if np.max(lfp_samples)>length(timestamps):
#     last_lfp_samp=np.where(lfp_samples<=length(timestamps),1,'last')
#     lfp_samples=lfp_samples(0:last_lfp_samp)



In [12]:
# lfp_timestamps=(lfp_samples/probe_lfp_sample_rate)-probe_shift

lfp_samples

array([        0,        12,        24, ..., 144332772, 144332784,
       144332796], dtype=int64)

In [ ]:
#how to align LFP timestamps??

In [7]:
# sync
ephysPath = r"Y:\DRpilot_649943_20230215\DRpilot_649943_20230215_probeABCDEF\Record Node 103\experiment1\recording1"
syncPath = r"Y:\DRpilot_649943_20230215\20230215T115233.h5"
syncDataset = sync.Dataset(syncPath)

syncBarcodeRising,syncBarcodeFalling = probeSync.get_sync_line_data(syncDataset,'barcode_ephys')
syncBarcodeTimes,syncBarcodes = ecephys.extract_barcodes_from_times(syncBarcodeRising,syncBarcodeFalling)


# ephys sync data
ephysSampleRate = 30000

dirNames = ['ProbeC','DAQ']

syncData = {key: {} for key in dirNames}
for key in syncData:
    keyMod = '*'+key+'-AP' if 'Probe' in key else '*'+key+'*'  
    datTimestampsPath = os.path.join(glob.glob(os.path.join(ephysPath,'continuous',keyMod))[0],'sample_numbers.npy')
    ttlStatesPath = os.path.join(glob.glob(os.path.join(ephysPath,'events',keyMod))[0],'TTL','states.npy')
    ttlTimestampsPath = os.path.join(os.path.dirname(ttlStatesPath),'sample_numbers.npy')

    datTimestamps = np.load(datTimestampsPath) / ephysSampleRate
    
    ttlStates = np.load(ttlStatesPath)
    ttlTimestamps = np.load(ttlTimestampsPath) / ephysSampleRate - datTimestamps[0]
    
    barcodeRising = ttlTimestamps[ttlStates>0]
    barcodeFalling = ttlTimestamps[ttlStates<0]
    barcodeTimes,barcodes = ecephys.extract_barcodes_from_times(barcodeRising,barcodeFalling)
    
    shift,sampleRate,endpoints = ecephys.get_probe_time_offset(syncBarcodeTimes,syncBarcodes,barcodeTimes,barcodes,0,ephysSampleRate)

    syncData[key]['shift'] = shift
    syncData[key]['sampleRate'] = sampleRate
    syncData[key]['LFPsampleRate'] = sampleRate/12

Master start index: [0]
Probe end index: -1
Master start index: [0]
Probe end index: -1


In [9]:
syncData['ProbeC']['sampleRate']/12

array([2500.02467129])

In [17]:
lfp_timestamps = lfp_samples/(syncData['ProbeC']['sampleRate']/12) - syncData['ProbeC']['shift']
lfp_timestamps

array([1.76363425e+01, 1.76411424e+01, 1.76459424e+01, ...,
       5.77501754e+04, 5.77501802e+04, 5.77501850e+04])

In [19]:
lfp_timestamps = lfp_samples/(syncData['ProbeC']['sampleRate']) - syncData['ProbeC']['shift']
lfp_timestamps

array([1.76363425e+01, 1.76411424e+01, 1.76459424e+01, ...,
       5.77501754e+04, 5.77501802e+04, 5.77501850e+04])